In [35]:
# Import base dependencies
import pandas as pd
import requests
import time
from datetime import datetime, timedelta
import math
import pyodbc

# Import config
import sys
import os

# Get the absolute path of the notebook's directory
notebook_dir = os.path.abspath('')
sys.path.append(os.path.join(notebook_dir, '..', 'src'))

# Now import the config file
from portfolio_manager.analytics import config
api_key = config.av_api_key

In [36]:
def load_access_table_to_df(access_file_path, table_name):
    """
    Loads a table from a Microsoft Access database into a Pandas DataFrame.

    Parameters:
    access_file_path (str): The full path to the Microsoft Access database file (.mdb or .accdb).
    table_name (str): The name of the table to load from the database.

    Returns:
    pd.DataFrame: The DataFrame containing the data from the specified table.

    Notes:
    - Requires the pyodbc library and the Microsoft Access Driver installed on your system.
    - Ensure the bit version (32-bit or 64-bit) of Python matches the installed Access Driver.
    """
    # Connection string for Microsoft Access
    conn_str = (
        r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};"
        r"DBQ=" + access_file_path + ";"
    )
    
    # Establish connection
    conn = pyodbc.connect(conn_str)
    
    # SQL query to select all data from the table
    query = f"SELECT * FROM {table_name}"
    
    # Read data into DataFrame
    df = pd.read_sql(query, conn)
    
    # Close the connection
    conn.close()
    
    return df

def create_db_dfs(table_name='brokerage'):
    # Path to database
    access_file_path = 'c:/users/culle/onedrive/documents/finances/investments.accdb'

    # Individual tables in database
    table_name = table_name
    
    return load_access_table_to_df(access_file_path,table_name)

def create_summary_dataframe_with_weighted_date(df, reference_date=None):
    """
    Creates a summary DataFrame by aggregating transaction data for each company (TICKER),
    including a weighted average purchase date based on total cost (quantity * purchase price).

    The summary includes:
    - TICKER: The company ticker.
    - ASSET_CLASS: The asset class (assumed consistent per TICKER, takes the first occurrence).
    - SECTOR: The sector (assumed consistent per TICKER, takes the first occurrence).
    - WEIGHTED_AVG_PURCHASE_DATE: The cost-weighted average purchase date.
    - TOTAL_COST: The total cost of all purchases.
    - TOTAL_QUANTITY: The total quantity of shares purchased.
    - AVG_PURCHASE_PRICE: The weighted average purchase price (total cost / total quantity).

    Parameters:
    df (pd.DataFrame): The input DataFrame with columns: ID, TICKER, ASSET_CLASS, SECTOR,
                      ACQUIRED, PURCHASE_PRICE, QUANTITY.
    reference_date (str): Reference date for date-to-numeric conversion (default: '2025-01-01').

    Returns:
    pd.DataFrame: The summary DataFrame with aggregated data per TICKER, including weighted purchase date.
    """
    # Set reference_date to current date if None
    if reference_date is None:
        reference_date = datetime.now().date()

    # Ensure ACQUIRED is in datetime format
    df['ACQUIRED'] = pd.to_datetime(df['ACQUIRED'])
    
    # Calculate the cost for each transaction
    df['COST'] = df['PURCHASE_PRICE'] * df['QUANTITY']
    
    # Convert ACQUIRED date to numeric (days since reference_date)
    reference_date = pd.to_datetime(reference_date)
    df['DAYS_SINCE_REF'] = (df['ACQUIRED'] - reference_date).dt.days
    
    # Calculate weighted days (cost * days since reference)
    df['WEIGHTED_DAYS'] = df['COST'] * df['DAYS_SINCE_REF']
    
    # Group by TICKER and aggregate
    summary_df = df.groupby('TICKER').agg({
        'ASSET_CLASS': 'first',
        'ALPHA_PICKED': 'first',
        'QUANT_RATING': 'first',
        'SECTOR': 'first',
        'ACQUIRED': 'min',
        'QUANTITY': 'sum',
        'COST': 'sum',
        'WEIGHTED_DAYS': 'sum'
    }).reset_index()
    
    # Rename columns for clarity
    summary_df = summary_df.rename(columns={
        'ACQUIRED': 'FIRST_ACQUIRED',
        'QUANTITY': 'TOTAL_QUANTITY',
        'COST': 'TOTAL_COST'
    })
    
    # Calculate weighted average purchase price
    summary_df['AVG_PURCHASE_PRICE'] = round(summary_df['TOTAL_COST'] / summary_df['TOTAL_QUANTITY'], ndigits=3)
    
    # Calculate weighted average days
    summary_df['WEIGHTED_AVG_DAYS'] = summary_df['WEIGHTED_DAYS'] / summary_df['TOTAL_COST']
    
    # Convert weighted average days back to a datetime
    summary_df['WEIGHTED_AVG_PURCHASE_DATE'] = reference_date + pd.to_timedelta(summary_df['WEIGHTED_AVG_DAYS'], unit='D')
    
    # Round the weighted average date to the nearest day for clarity
    summary_df['WEIGHTED_AVG_PURCHASE_DATE'] = summary_df['WEIGHTED_AVG_PURCHASE_DATE'].dt.round('D')
    
    # Reorder columns as requested
    columns_order = [
        'TICKER', 'ALPHA_PICKED', 'QUANT_RATING', 'ASSET_CLASS', 'SECTOR', 'WEIGHTED_AVG_PURCHASE_DATE',
        'TOTAL_COST', 'TOTAL_QUANTITY', 'AVG_PURCHASE_PRICE'
    ]
    summary_df = summary_df[columns_order]
    
    # Drop temporary columns from input DataFrame
    df.drop(['COST', 'DAYS_SINCE_REF', 'WEIGHTED_DAYS'], axis=1, inplace=True)
    
    return summary_df

In [37]:
# Load database tables
r_analysis = create_db_dfs('retirement')
alpha_raw = create_db_dfs('alpha_picks')
watch_list = create_db_dfs('watch_list')
b_analysis = create_db_dfs('brokerage')
york_401k_list = create_db_dfs('york_401k')
joint_wros = create_db_dfs('joint_wros')

C:\Users\culle\AppData\Local\Temp\ipykernel_15980\47529893.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [38]:
# Merge portfolios and watch lists to create master portfolio
r_analysis2 = pd.merge(
        r_analysis,
        alpha_raw, 
        on='TICKER',
        how='outer'
    )
r_analysis2.drop(columns=['ID_x','ID_y'], inplace=True)

alpha_plus_watch = pd.concat([alpha_raw, watch_list], ignore_index=True)

b_analysis2 = pd.merge(
    b_analysis,
    alpha_plus_watch,
    on='TICKER',
    how='outer'
)
b_analysis2.drop(columns=['ID_x','ID_y'], inplace=True)

joint_raw = pd.merge(
    joint_wros,
    alpha_raw,
    on='TICKER',
    how='outer',
)
joint_raw.drop(columns=['ID_x', 'ID_y'], inplace=True)

In [39]:
# Create summary tables of portfolios
brokerage_df = create_summary_dataframe_with_weighted_date(b_analysis2)
ira_df = create_summary_dataframe_with_weighted_date(r_analysis2)
#joint_df = create_summary_dataframe_with_weighted_date(joint_raw)

In [40]:
def get_history(symbol, api_key, days=252):  # ~1 year default
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}&outputsize=full&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if "Time Series (Daily)" not in response:
            error_msg = response.get('Note', response.get('Information', 'Unknown error'))
            print(f"Error fetching price data for {symbol}: {error_msg}")
            print(f"Response keys: {list(response.keys())}")
            return None
        
        time_series = response["Time Series (Daily)"]
        df = pd.DataFrame.from_dict(time_series, orient="index", dtype=float)
        
        # Debug: Print available columns
        # print(f"Columns for {symbol}: {list(df.columns)}")
        
        # Rename columns dynamically
        column_map = {
            col: name for col, name in [
                ("1. open", "Open"), ("2. high", "High"), ("3. low", "Low"),
                ("4. close", "Close"), ("5. volume", "Volume"), ("6. volume", "Volume"),
                ("7. adjusted close", "Adjusted Close"), ("8. dividend amount", "Dividend")
            ] if col in df.columns
        }
        if "5. volume" not in df.columns and "6. volume" not in df.columns:
            print(f"No volume data for {symbol}")
            return None
        
        df = df.rename(columns=column_map)
        df.index = pd.to_datetime(df.index)
        df = df.sort_index().tail(days)
        return df
    except Exception as e:
        print(f"Exception fetching price data for {symbol}: {str(e)}")
        return None

def get_fundamentals(symbol, api_key, current_price):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if not response or "Symbol" not in response:
            error_msg = response.get('Note', response.get('Information', 'No data'))
            print(f"Error fetching fundamentals for {symbol}: {error_msg}")
            print(f"Full response: {response}")
            return None
        
        def safe_float(value, default):
            if value in [None, 'None', '']:
                return default
            try:
                return float(value)
            except (ValueError, TypeError):
                return default
        
        pe_ratio = safe_float(response.get('PERatio'), float('inf'))
        pb_ratio = safe_float(response.get('PriceToBookRatio'), float('inf'))
        
        # Calculate EPS and Book Value
        eps = current_price / pe_ratio if pe_ratio != float('inf') and pe_ratio != 0 else 0
        book_value = current_price / pb_ratio if pb_ratio != float('inf') and pb_ratio != 0 else 0
        
        fundamentals = {
            'pe_ratio': pe_ratio,
            'pb_ratio': pb_ratio,
            'dividend_yield': safe_float(response.get('DividendYield'), 0),
            'debt_to_equity': safe_float(response.get('DebtToEquityRatio'), float('inf')),
            'eps': eps,
            'book_value': book_value
        }
        
        return fundamentals
    except Exception as e:
        print(f"Exception fetching fundamentals for {symbol}: {str(e)}")
        return None

def calculate_vwap(df, days=126):  # ~2 quarters
    if 'Volume' not in df.columns:
        print("Missing Volume column in DataFrame")
        return None
    
    # Calculate VWAPS for designated time period
    vwap_analysis = df[-days:].copy()
    vwap_analysis['Cumulative_LTPV'] = (vwap_analysis['Low'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_HTPV'] = (vwap_analysis['High'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_Volume'] = vwap_analysis['Volume'].cumsum()
    vwap_analysis['Entry'] = round(vwap_analysis['Cumulative_LTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    vwap_analysis['Exit'] = round(vwap_analysis['Cumulative_HTPV'] / vwap_analysis['Cumulative_Volume'], 2)

    # Return VWAPS
    return vwap_analysis[-1:].copy()

def build_analysis_table(summary_df, api_key, margin_of_safety=0.9, vwap_days=126, graham_margin=0.95):
    """
    Builds a portfolio analysis table using financial data from Alpha Vantage API,
    with ticker symbols extracted from the provided summary DataFrame.

    Parameters:
    summary_df (pd.DataFrame): Summary DataFrame with at least a 'TICKER' column.
    api_key (str): Alpha Vantage API key for fetching financial data.
    margin_of_safety (float): Margin of safety for VWAP buy threshold (default: 0.9).
    vwap_days (int): Number of days for VWAP calculation (default: 126).
    graham_margin (float): Margin for Graham buy threshold (default: 0.95).

    Returns:
    list: List of lists containing analysis data for each ticker:
          [symbol, market_price, buy_threshold, graham_buy_threshold, exit_price,
           pe_ratio, pb_ratio, dividend_yield, decision]
    """
    # Extract unique tickers from summary_df
    ticker_symbols = [{'symbol': ticker, 'is_etf': False} for ticker in summary_df['TICKER'].unique()]
    
    portfolio = []
    etf_list = ['FBTC', 'ITA', 'SCHF', 'SCHE','SCHX', 'AOA', 'AOK', 'AOM',
                'AOR', 'BLV', 'SCHA', 'SCHD', 'SCHH', 'SCHM', 'SCHP', 'SCHR',
                'SCHZ']
    
    for ticker in ticker_symbols:
        symbol = ticker['symbol']
        is_etf = symbol in etf_list
        
        # Get price data
        raw_data = get_history(symbol, api_key)
        if raw_data is None:
            portfolio.append([symbol, None, None, None, None, None, None, None, "Error"])
            continue
        
        # Get fundamentals (skip for ETFs)
        current_price = raw_data['Close'].iloc[-1]  # Use Close for fundamental calcs
        fundamentals = None if is_etf else get_fundamentals(symbol, api_key, current_price)
        if not is_etf and fundamentals is None:
            portfolio.append([symbol, None, None, None, None, None, None, None, "Error"])
            continue
        
        # Calculate VWAP
        vwap_data = calculate_vwap(raw_data, days=vwap_days)
        if vwap_data is None:
            portfolio.append([symbol, None, None, None, None, None, None, None, "Error"])
            continue
        
        # Extract data
        market_price = round(raw_data['Close'].iloc[-1], 2)  # Use Close for buys
        entry_price = round(vwap_data['Entry'].iloc[0], 2)
        # one_month_low = round(vwap_data['1mo_Low'].iloc[0], 2)
        exit_price = round(vwap_data['Exit'].iloc[0], 2)
        # one_month_high = round(vwap_data['1mo_High'].iloc[0], 2)
        buy_threshold = round(entry_price * margin_of_safety, 2)  # 10% margin

        # Graham buy threshold (for stocks only)
        graham_buy_threshold = None
        if not is_etf:
            if fundamentals['eps'] > 0 and fundamentals['book_value'] > 0:
                # Calculate desired price where P/E × P/B = 38
                desired_price = math.sqrt(38 * fundamentals['eps'] * fundamentals['book_value'])
                graham_buy_threshold = round(desired_price * graham_margin, 2)  # 5% margin
            else:
                graham_buy_threshold = buy_threshold  # Default to VWAP threshold
        
        # Volume filter: 20% of 21-day average
        avg_volume = raw_data['Volume'][-21:].mean()
        today_volume = raw_data['Volume'].iloc[-1]
        volume_ok = today_volume >= avg_volume * 0.2
        
        # Graham's fundamental checks (for stocks only)
        graham_ok = True
        if not is_etf:
            graham_ok = (
                (fundamentals['pe_ratio'] < 19 and fundamentals['pb_ratio'] < 2.0) or
                (fundamentals['pe_ratio'] * fundamentals['pb_ratio'] < 38 and 
                 fundamentals['pe_ratio'] < 100 and fundamentals['pb_ratio'] < 10)
                 ) and fundamentals['dividend_yield'] >= 0 and fundamentals['debt_to_equity'] < 2
        
        # Decision logic
        decision = "Hold"
        if market_price <= min(buy_threshold, graham_buy_threshold or float('inf')) and volume_ok and graham_ok:
            decision = "Buy"
        elif market_price >= exit_price and volume_ok:
            decision = "Sell"
        
        # Prepare fundamentals for output
        pe_ratio = None if is_etf else fundamentals['pe_ratio']
        pb_ratio = None if is_etf else fundamentals['pb_ratio']
        dividend_yield = None if is_etf else fundamentals['dividend_yield']
        
        portfolio.append([
            symbol, market_price, buy_threshold, graham_buy_threshold, exit_price,
            pe_ratio, pb_ratio, dividend_yield, decision
        ])

        # Convert portfolio list to DataFrame with specified column names
        portfolio_df = pd.DataFrame(portfolio, columns=[
            'TICKER', 'PRICE', 'VWAP_LOW', 'BUY_PRICE', 'SELL_PRICE',
            'P/E', 'P/B', 'DivYield', 'RATING'
            ])
        
        # Minimal delay for server stability (75 calls/minute = ~0.8 seconds/call)
        time.sleep(0.1)
    
    return portfolio_df

def build_portfolio_df(portfolio_df, api_key, historical_alpha, 
                       historical_return, inception_date, price_limit=False, 
                       margin_of_safety=.9, vwap_days=63, graham_margin=0.9, 
                       desired_total_exposure=0.9, cash_pos=0,
                       run_date=None):

    if run_date is None:
        run_date = datetime.now().date()
    run_date = pd.to_datetime(run_date)

    # Get price data and analysis
    price_data = build_analysis_table(portfolio_df, api_key, margin_of_safety, vwap_days, graham_margin)

    # Merge portfolio holdings with price data for each position
    expanded_portfolio_df = pd.merge(
            portfolio_df,
            price_data, 
            on='TICKER',
            how='left'
        )
    
    # Filter portfolio dataframe for desired allocation
    if price_limit:
        expanded_portfolio_df = expanded_portfolio_df[(expanded_portfolio_df['TOTAL_QUANTITY'] > 0) | 
                                                      (expanded_portfolio_df['PRICE'] <= 100)]
    else:
        pass
    quant_rankings = list(expanded_portfolio_df['QUANT_RATING'])
    quant_rankings.sort(reverse=True)
    if price_limit:
        quant_threshold = quant_rankings[14]
    else:
        quant_threshold = quant_rankings[19]
    portfolio_df_filtered = expanded_portfolio_df[(expanded_portfolio_df['TOTAL_QUANTITY'] > 0) | 
                                                  (expanded_portfolio_df['QUANT_RATING'] >= quant_threshold)]
    portfolio_df_sorted = portfolio_df_filtered.sort_values(by='QUANT_RATING', 
                                                            ascending=False).reset_index(drop=True)

    # Calculate total value of each held position
    portfolio_df_sorted['VALUE'] = round(portfolio_df_sorted['TOTAL_QUANTITY'] * portfolio_df_sorted['PRICE'], ndigits=2)

    # Calculate value of portfolio, portfolio weights, total return and CAGR for each position
    portfolio_total = portfolio_df_sorted['VALUE'].sum() + cash_pos
    portfolio_df_sorted['PW%'] = round(portfolio_df_sorted['VALUE'] / portfolio_total * 100, ndigits=2)
    portfolio_df_sorted.loc[portfolio_df_sorted['TOTAL_COST'] < 0, 'AVG_PURCHASE_PRICE'] = 0.01
    portfolio_df_sorted['TOTAL_RETURN'] = portfolio_df_sorted['VALUE'] - portfolio_df_sorted['TOTAL_COST']
    portfolio_df_sorted['ROI'] = round(portfolio_df_sorted['TOTAL_RETURN'] / portfolio_df_sorted['TOTAL_COST'] * 100, ndigits=4)
    portfolio_df_sorted.loc[portfolio_df_sorted['AVG_PURCHASE_PRICE'] == 0.010, 'ROI'] = float('inf')

    # Calculate desired target price to optimize return
    portfolio_days_held = (run_date - inception_date).days
    portfolio_years_held = portfolio_days_held / 360
    portfolio_cagr = (1+historical_return)**(1/portfolio_years_held)-1
    alpha_age = (run_date.date() - pd.to_datetime("07-01-2022", format="%m-%d-%Y").date()).days / 360
    alpha_return = (1+historical_alpha)**(1/alpha_age)-1
    desired_return = min(portfolio_cagr, alpha_return)
    portfolio_df_sorted['YEARS_HELD'] = ((run_date - portfolio_df_sorted['WEIGHTED_AVG_PURCHASE_DATE']).dt.days) / 360
    portfolio_df_sorted['TARGET'] = round(portfolio_df_sorted['AVG_PURCHASE_PRICE'] * (1+desired_return)**portfolio_df_sorted['YEARS_HELD'], ndigits=2)
    portfolio_df_sorted['CAGR'] = round(((1+portfolio_df_sorted['ROI']/100) ** (1/portfolio_df_sorted['YEARS_HELD']) - 1) * 100, ndigits=2)

    # Calculate quant-based portfolio allocation and position adjustments to meet desired allocation
    desired_position_size = portfolio_total * desired_total_exposure * (1/20)
    portfolio_df_sorted['DESIRED_POS'] = desired_position_size * portfolio_df_sorted['QUANT_RATING'] / 5
    portfolio_df_sorted.loc[portfolio_df_sorted['QUANT_RATING'] < quant_threshold, 'DESIRED_POS'] = 0
    portfolio_df_sorted['POS_ADJUSTMENT'] = round((portfolio_df_sorted['DESIRED_POS'] - portfolio_df_sorted['VALUE']) / portfolio_df_sorted['PRICE'], ndigits=0)

    # Sorta final portfolio table
    final_portfolio_df = portfolio_df_sorted.sort_values(by='PW%', ascending=False).reset_index(drop=True)

    return final_portfolio_df

def buys_and_sells_tables(portfolio_df):
    # Build action table
    action_columns = ['TICKER', 'QUANT_RATING', 'TOTAL_QUANTITY', 'POS_ADJUSTMENT', 'PRICE', 
                    'VWAP_LOW', 'BUY_PRICE', 'SELL_PRICE', 'TARGET', 'RATING']
    portfolio_df['ACTION'] = abs(portfolio_df['POS_ADJUSTMENT'] / portfolio_df['TOTAL_QUANTITY']) > 0.2
    actions_df = portfolio_df.loc[portfolio_df['ACTION']==True]
    actions_df = actions_df[action_columns]

    # Retrieve short-term price data
    short_term_vwaps = build_analysis_table(actions_df, api_key, margin_of_safety=0.9999, vwap_days=21)
    short_term_columns = ['TICKER', 'VWAP_LOW', 'SELL_PRICE']
    one_month = short_term_vwaps[short_term_columns].copy()
    one_month['RANGE'] = one_month['SELL_PRICE'] - one_month['VWAP_LOW']
    one_month['VOLATILITY'] = round(one_month['RANGE'] / one_month['VWAP_LOW'], ndigits=4) * 100
    one_month.rename(columns={'VWAP_LOW': '1mo_Low', 'SELL_PRICE': '1mo_High'}, inplace=True)
    one_month = one_month.sort_values('VOLATILITY', ascending=False, ignore_index=True)

    # Merge one-month price data with actions table
    actions_plus_df = pd.merge(actions_df, one_month, on='TICKER', how='left')
    actions_plus_df['WEIGHTED_RATING'] = actions_plus_df['QUANT_RATING'] * (1+actions_plus_df['VOLATILITY'])
    actions_plus_df = actions_plus_df.sort_values(by='WEIGHTED_RATING', 
                                                  ascending=False).reset_index(drop=True).copy()


    # Build buy and sell tables
    buys_df = actions_plus_df.loc[actions_plus_df['POS_ADJUSTMENT'] > 0]
    sells_df = actions_plus_df.loc[actions_plus_df['POS_ADJUSTMENT'] < 0]
    buys_columns = ['TICKER', 'POS_ADJUSTMENT', 'PRICE', 'VWAP_LOW', '1mo_Low', 'BUY_PRICE', 'RATING']
    sells_columns = ['TICKER', 'POS_ADJUSTMENT', 'PRICE', 'SELL_PRICE', '1mo_High', 'TARGET', 'RATING']
    buys_final = buys_df[buys_columns].copy()
    sells_final = sells_df[sells_columns].copy()

    return buys_final, sells_final

In [41]:
final_portfolio_columns = ['TICKER', 'QUANT_RATING', 'TOTAL_QUANTITY', 'AVG_PURCHASE_PRICE', 
                           'TOTAL_COST', 'PRICE', 'VALUE', 'TOTAL_RETURN', 'ROI']

In [42]:
alpha_historical_return = 2.1

ira_cash = 47980.57
ira_historical_return = 0.9995

brokerage_cash = 7932.82
brokerage_historical_return = 0.4687

joint_cash = 13000
joint_historical_return = 0

york_401k_cash = 2995.76

In [43]:
ira_portfolio_df = build_portfolio_df(ira_df, api_key, margin_of_safety=.975, vwap_days=63, 
                                      historical_return=ira_historical_return, 
                                      historical_alpha=alpha_historical_return, 
                                      inception_date=pd.to_datetime("11-15-2023", format="%m-%d-%Y"),
                                      cash_pos=ira_cash)
time.sleep(60)
ira_buys_final, ira_sells_final = buys_and_sells_tables(ira_portfolio_df)

In [44]:
ira_buys, ira_sells = buys_and_sells_tables(ira_portfolio_df)

In [45]:
round(ira_portfolio_df['VALUE'].sum() + ira_cash, ndigits=2)

145760.55

In [46]:
ira_buys

,TICKER,POS_ADJUSTMENT,PRICE,VWAP_LOW,1mo_Low,BUY_PRICE,RATING
0,COMM,119.0,15.61,9.73,13.26,2.08,Sell
1,CLS,9.0,188.92,152.51,188.87,48.20,Sell
2,POWL,22.0,262.95,203.66,234.68,156.27,Sell
3,STRL,23.0,279.58,231.20,274.27,91.64,Sell
4,BLBD,101.0,55.31,45.48,51.50,27.72,Sell
6,SSRM,382.0,17.11,12.80,14.59,19.87,Sell
9,LRN,7.0,165.17,138.11,144.88,77.65,Sell
11,SKYW,22.0,119.08,104.22,112.22,145.20,Sell
12,EZPW,73.0,16.00,13.47,14.70,25.82,Sell
13,CCL,133.0,31.30,25.94,29.13,22.28,Sell


In [47]:
ira_sells

,TICKER,POS_ADJUSTMENT,PRICE,SELL_PRICE,1mo_High,TARGET,RATING
5,APP,-8.0,441.68,392.84,427.72,203.46,Sell
7,CRDO,-20.0,114.04,93.44,116.18,0.01,Sell
8,ARQT,-498.0,15.98,14.83,15.68,18.32,Sell
10,OKTA,-67.0,92.05,102.39,96.99,96.30,Hold
18,QTWO,-61.0,79.52,85.72,80.11,80.80,Hold
20,PYPL,-57.0,69.90,72.78,71.08,74.16,Hold
21,WFC,-41.0,79.10,79.28,79.88,72.07,Hold


In [48]:
ira_portfolio_df[final_portfolio_columns]

,TICKER,QUANT_RATING,TOTAL_QUANTITY,AVG_PURCHASE_PRICE,TOTAL_COST,PRICE,VALUE,TOTAL_RETURN,ROI
0,ARQT,3.20,498.0,15.932,7934.2500,15.98,7958.04,23.7900,0.2998
1,ATGE,4.62,52.0,116.372,6051.3625,134.88,7013.76,962.3975,15.9038
2,OKTA,4.23,67.0,91.134,6105.9460,92.05,6167.35,61.4040,1.0056
3,DXPE,4.74,49.0,105.590,5173.8900,124.89,6119.61,945.7200,18.2787
4,WLDN,4.98,55.0,36.080,1984.3920,111.11,6111.05,4126.6580,207.9558
5,EAT,4.33,39.0,124.038,4837.4853,150.40,5865.60,1028.1147,21.2531
6,EZPW,4.75,316.0,15.460,4885.3600,16.00,5056.00,170.6400,3.4929
7,LRN,4.63,30.0,107.868,3236.0438,165.17,4955.10,1719.0562,53.1222
8,QTWO,3.01,61.0,79.148,4828.0223,79.52,4850.72,22.6977,0.4701
9,CLS,4.93,25.0,0.010,-86.3962,188.92,4723.00,4809.3962,inf


In [49]:
time.sleep(300)

In [50]:
brokerage_portfolio_df = build_portfolio_df(brokerage_df, api_key, margin_of_safety=.975, vwap_days=42, 
                                      historical_return=ira_historical_return, price_limit=True,
                                      historical_alpha=alpha_historical_return, 
                                      inception_date=pd.to_datetime("12-26-2023", format="%m-%d-%Y"),
                                      cash_pos=brokerage_cash)
time.sleep(60)
brokerage_buys, brokerage_sells = buys_and_sells_tables(brokerage_portfolio_df)


In [51]:
round(brokerage_portfolio_df['VALUE'].sum() + brokerage_cash, ndigits=2)

27498.87

In [52]:
brokerage_buys

,TICKER,POS_ADJUSTMENT,PRICE,VWAP_LOW,1mo_Low,BUY_PRICE,RATING
1,BKTI,17.0,72.89,47.10,51.16,33.96,Sell
2,COMM,16.0,15.61,11.16,13.26,2.08,Sell
3,LX,158.0,6.75,6.45,6.36,20.20,Hold
4,BLBD,19.0,55.31,47.42,51.50,27.72,Sell
5,SSRM,19.0,17.11,13.32,14.59,19.87,Sell
7,GFI,39.0,31.12,25.48,28.13,16.27,Sell
8,NGD,74.0,5.54,4.51,4.64,3.03,Sell
11,CCL,36.0,31.30,27.88,29.13,22.28,Sell
12,UBER,5.0,96.79,88.19,89.22,45.57,Sell
13,SYF,16.0,73.20,67.44,69.78,106.37,Sell


In [53]:
brokerage_sells

,TICKER,POS_ADJUSTMENT,PRICE,SELL_PRICE,1mo_High,TARGET,RATING
0,PSIX,-10.0,91.26,88.50,95.91,83.47,Sell
6,ARQT,-83.0,15.98,15.28,15.68,18.58,Sell
9,OKTA,-40.0,92.05,97.11,96.99,97.94,Hold
10,EZPW,-67.0,16.00,14.67,15.19,17.25,Sell
14,FBTC,-5.0,101.79,101.88,102.73,97.76,Hold
15,PPC,-19.0,46.16,47.45,48.86,46.37,Hold
16,PYPL,-16.0,69.90,72.89,71.08,72.38,Hold
17,WFC,-8.0,79.10,81.03,79.88,83.32,Hold


In [54]:
brokerage_portfolio_df[final_portfolio_columns]

,TICKER,QUANT_RATING,TOTAL_QUANTITY,AVG_PURCHASE_PRICE,TOTAL_COST,PRICE,VALUE,TOTAL_RETURN,ROI
0,OKTA,4.23,40.0,95.088,3803.500000,92.05,3682.00,-121.500000,-3.1944
1,EZPW,4.75,140.0,15.773,2208.153384,16.00,2240.00,31.846616,1.4422
2,PSIX,4.98,24.0,82.730,1985.520000,91.26,2190.24,204.720000,10.3106
3,FBTC,4.47,16.0,86.612,1385.790000,101.79,1628.64,242.850000,17.5243
4,ARQT,3.20,83.0,15.996,1327.643700,15.98,1326.34,-1.303700,-0.0982
5,PGY,4.96,39.0,18.246,711.590000,33.76,1316.64,605.050000,85.0279
6,PYPL,3.37,16.0,68.292,1092.669800,69.90,1118.40,25.730200,2.3548
7,FINV,4.90,113.0,8.920,1007.930000,9.43,1065.59,57.660000,5.7206
8,COMM,4.99,63.0,0.010,-463.431000,15.61,983.43,1446.861000,inf
9,SSRM,4.98,53.0,12.210,647.130000,17.11,906.83,259.700000,40.1310


In [55]:
# Import 401k options from database
york_401k_sorted = york_401k_list.sort_values(by='QUANT_RATING', ascending=False).reset_index(drop=True)
york_401k_preferred = york_401k_sorted[:5]
fouroh1k_columns = ['TICKER', 'PRICE', 'VWAP_LOW', 'SELL_PRICE', 'RATING']
# Fetch price data for ETFs available in 401k
fouroh1k_price_data = build_analysis_table(york_401k_preferred, api_key, margin_of_safety=.999, vwap_days=32)
# Reset purchase rating based on simplified data
fouroh1k_price_data.loc[fouroh1k_price_data['PRICE']<fouroh1k_price_data['VWAP_LOW'], 'RATING'] = "Buy"

In [56]:
fouroh1k_price_data[fouroh1k_columns]

,TICKER,PRICE,VWAP_LOW,SELL_PRICE,RATING
0,ITA,195.81,193.69,196.81,Hold
1,SCHF,23.07,22.32,22.47,Sell
2,SCHE,31.90,30.79,31.00,Sell
3,SCHX,25.56,24.95,25.18,Sell
4,AOA,85.97,83.54,84.18,Sell


In [57]:
'''
joint_portfolio_df = build_portfolio_df(joint_df, api_key, margin_of_safety=.999, vwap_days=63, 
                                      historical_return=joint_historical_return,
                                      historical_alpha=alpha_historical_return, 
                                      inception_date=pd.to_datetime("8-25-2023", format="%m-%d-%Y"),
                                      cash_pos=joint_cash)
time.sleep(60)
joint_buys, joint_sells = buys_and_sells_tables(joint_df)
'''

'\njoint_portfolio_df = build_portfolio_df(joint_df, api_key, margin_of_safety=.999, vwap_days=63, \n                                      historical_return=joint_historical_return,\n                                      historical_alpha=alpha_historical_return, \n                                      inception_date=pd.to_datetime("8-25-2023", format="%m-%d-%Y"),\n                                      cash_pos=joint_cash)\ntime.sleep(60)\njoint_buys, joint_sells = buys_and_sells_tables(joint_df)\n'

In [58]:
joint_new = build_analysis_table(joint_raw, api_key, margin_of_safety=.999, vwap_days=21)

In [59]:
joint_raw_columns = ['TICKER', 'QUANT_RATING']
joint_raw = joint_raw[joint_raw_columns]
joint_est = pd.merge(
    joint_raw,
    joint_new,
    on='TICKER',
    how='outer'
)
joint_est = joint_est.loc[joint_est['QUANT_RATING'] >= 4.50]
joint_est['POS_SIZE'] = round((joint_cash * 0.9 * (1/20)) / joint_est['VWAP_LOW'], ndigits=0)

In [60]:
joint_est_columns = ['TICKER', 'QUANT_RATING', 'PRICE', 'POS_SIZE', 'VWAP_LOW', 'BUY_PRICE']
joint_est[joint_est_columns].sort_values(by='QUANT_RATING', ascending=False).reset_index(drop=True)

,TICKER,QUANT_RATING,PRICE,POS_SIZE,VWAP_LOW,BUY_PRICE
0,COMM,4.99,15.61,44.0,13.25,2.20
1,SSRM,4.98,17.11,40.0,14.58,20.98
2,WLDN,4.98,111.11,6.0,97.21,40.99
3,TSM,4.96,232.99,2.0,235.29,98.79
4,CLS,4.93,188.92,3.0,188.70,50.87
5,CCL,4.92,31.30,20.0,29.10,23.52
6,ALL,4.91,205.52,3.0,201.04,241.42
7,STRL,4.90,279.58,2.0,274.03,96.73
8,SYF,4.87,73.20,8.0,69.72,112.28
9,TMUS,4.80,251.95,2.0,244.24,136.98


In [61]:
'''def setup_driver():
    chrome_options = Options()
    chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")  # Connect to existing Chrome
    driver = webdriver.Chrome(options=chrome_options)  # No need for user-data-dir here
    return driver

def scrape_quant_ratings(symbol, driver, days=63):
    url = f'https://seekingalpha.com/symbol/{symbol}/ratings/quant-ratings'
    driver.get(url)
    time.sleep(5)  # Extra delay for dynamic content
    
    try:
        # Wait for table to load
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
    except Exception as e:
        driver.save_screenshot(f'error_{symbol}.png')
        with open(f'page_source_{symbol}.html', 'w', encoding='utf-8') as f:
            f.write(driver.page_source)
        print(f"Page load failed for {symbol}: {e}. Saved screenshot and page source for debugging.")
        raise Exception(f"Failed to load page for {symbol}")
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Find the quant rating history table
    table = soup.find('table')
    
    if not table:
        raise Exception("Table not found")
    
    rows = table.find_all('tr')[1:]  # Skip header
    data = []
    for row in rows:
        cols = row.find_all('td')
        if len(cols) >= 4:
            date_str = cols[0].text.strip()
            try:
                date = datetime.strptime(date_str, '%m/%d/%Y')
            except ValueError:
                continue
            quant_score_str = cols[3].text.strip()
            try:
                quant_score = float(quant_score_str)
            except ValueError:
                quant_score = np.nan
            data.append({'date': date, 'quant_score': quant_score})
    
    df = pd.DataFrame(data)
    df = df.sort_values('date', ascending=False)
    
    # Filter last 63 days (current date as 02:24 PM MDT, Aug 10, 2025)
    end_date = datetime(2025, 8, 10, 14, 24)  # 2:24 PM MDT
    start_date = end_date - timedelta(days=days)
    df_filtered = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    
    return df_filtered

def compute_stats(df):
    scores = df['quant_score'].dropna()
    if len(scores) == 0:
        return {'average': np.nan, 'median': np.nan}
    return {
        'average': scores.mean(),
        'median': scores.median()
    }

# Main function
def main(companies):
    driver = setup_driver()
    try:
        results = {}
        for symbol in companies:
            try:
                df = scrape_quant_ratings(symbol, driver)
                stats = compute_stats(df)
                results[symbol] = stats
            except Exception as e:
                print(f"Error for {symbol}: {e}")
                results[symbol] = {'average': np.nan, 'median': np.nan}
        
        # Output results
        results_df = pd.DataFrame.from_dict(results, orient='index')
        print(results_df)
    finally:
        driver.quit()

# Example usage
companies = ['DXPE', 'AAPL', 'GOOG']  # Replace with your list
main(companies)
'''

'def setup_driver():\n    chrome_options = Options()\n    chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")  # Connect to existing Chrome\n    driver = webdriver.Chrome(options=chrome_options)  # No need for user-data-dir here\n    return driver\n\ndef scrape_quant_ratings(symbol, driver, days=63):\n    url = f\'https://seekingalpha.com/symbol/{symbol}/ratings/quant-ratings\'\n    driver.get(url)\n    time.sleep(5)  # Extra delay for dynamic content\n    \n    try:\n        # Wait for table to load\n        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.TAG_NAME, \'table\')))\n    except Exception as e:\n        driver.save_screenshot(f\'error_{symbol}.png\')\n        with open(f\'page_source_{symbol}.html\', \'w\', encoding=\'utf-8\') as f:\n            f.write(driver.page_source)\n        print(f"Page load failed for {symbol}: {e}. Saved screenshot and page source for debugging.")\n        raise Exception(f"Failed to load page for {symbol

In [62]:
# Machine Learning

In [63]:
# Data processing and clearning
# Must be in numpy array or tf.Dataset object format

In [64]:
# Feature selection and normalization

In [65]:
# Build model

In [66]:
# Train model

In [67]:
# Evaluate model

In [68]:
# Refine model through hyperparameter tuning